In [1]:
import tensorflow.keras,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [26]:
body_class = cv.CascadeClassifier("D:/jb_python/FinalProject/zz/haarcascade_upperbody.xml")

In [27]:
def body_extractor(photo):
    gphoto = cv.cvtColor(photo, cv.COLOR_BGR2GRAY)
    detected = body_class.detectMultiScale(gphoto)

    if detected == ():
        return None
    else : 
        (x, y, w, h) = detected[0]
        cphoto = photo[y:y+h, x:x+w]
        return cphoto

In [11]:
name = input("What the pose : ")
directory = name
parent_dir = "D:/jb_python/FinalProject/dataset/face_train/"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
parent_dir = "D:/jb_python/FinalProject/dataset/face_test/"
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [30]:
load_path = "./dataset/forw_0828/"
save_path = "./dataset/face_train/fo/"

for img in os.listdir(load_path):
    aa = load_path + img
    photo = cv.imread(aa, cv.IMREAD_COLOR)
    if body_extractor(photo) is not None:
        cphoto = body_extractor(photo)
        body = cv.resize(cphoto, (224,224))

        cv.imwrite(save_path+img, body)
    else:
        print("body not found")
        pass

body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not f

C:\Users\marbi\AppData\Local\Temp/ipykernel_34156/1629830274.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if detected == ():



body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not found
body not 

In [14]:
load_path = "./dataset/vali2/co/"
save_path = "./dataset/face_test/co/"

for img in os.listdir(load_path):
    aa = load_path + img
    photo = cv.imread(aa, cv.IMREAD_COLOR)
    if face_extractor(photo) is not None:
        cphoto = face_extractor(photo)
        face = cv.resize(cphoto, (224, 224))
        # face = cv.cvtColor(face, cv.COLOR_GRAY2BGR)

        cv.imwrite(save_path+img, face)
    else:
        print("Face not found")
        pass

C:\Users\marbi\AppData\Local\Temp/ipykernel_3088/3621784647.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if detected == ():


Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found


In [15]:
img_shape = (224, 224, 3)
model = VGG16(weights="imagenet", include_top=False, input_shape=img_shape)

In [16]:
def layer_adder(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation="relu")(top_model)
    top_model = Dense(512, activation="relu")(top_model)
    top_model = Dense(num_classes, activation="sigmoid")(top_model)
    return top_model

In [17]:
num_classes = 1
FC_Head = layer_adder(model, num_classes)

model = Model(inputs=model.input, outputs = FC_Head)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
###########################################################
# 데이터 전처리
###########################################################


train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=45, width_shift_range=0.3, height_shift_range=0.3, horizontal_flip=True, fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/face_train"
validation_dir = './dataset/face_test'

batch_size = 16
# batch_size = 64


train_generator = train_datagen.flow_from_directory( \
        train_dir,                  # 타깃 디렉터리
        target_size=(224, 224),     # 모든 이미지를 224 × 224 크기
        batch_size=batch_size,
        class_mode='binary')        #

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)  #(8, 224, 224, 3)
    print('배치 레이블 크기:', labels_batch.shape) #(8,)
    break

Found 1240 images belonging to 2 classes.
Found 208 images belonging to 2 classes.
배치 데이터 크기: (16, 224, 224, 3)
배치 레이블 크기: (16,)


In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("face_detector.h15", monitor="val_loss", mode="min", save_best_only=True, verbose=1)

earlystop = EarlyStopping(monitor="val_loss", min_delta=0, patience = 3, verbose = 1, restore_best_weights=True)

callbacks = [earlystop, checkpoint]

model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
steps_per_epochs = 60 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 15

history = model.fit(
      train_generator ,
      steps_per_epoch = steps_per_epochs,
      epochs=epochs ,
      callbacks = callbacks, 
      validation_data=validation_generator,
      validation_steps=13)

Epoch 1/15
60/60 [==============================] - 48s 620ms/step - loss: 0.6935 - accuracy: 0.4790 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 0.69315, saving model to face_detector.h15
INFO:tensorflow:Assets written to: face_detector.h15\assets
Epoch 2/15
60/60 [==============================] - 32s 531ms/step - loss: 0.6932 - accuracy: 0.5095 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00002: val_loss improved from 0.69315 to 0.69315, saving model to face_detector.h15
INFO:tensorflow:Assets written to: face_detector.h15\assets
Epoch 3/15
60/60 [==============================] - 33s 539ms/step - loss: 0.6932 - accuracy: 0.5052 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 0.69315
Epoch 4/15
60/60 [==============================] - 33s 546ms/step - loss: 0.6933 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00004: val_loss did not improve from 0.69315
Epoch 5/15
60/60 [========

In [8]:
model.save('./tn_model/add_yw_tm_model.h5')

D:\Program Files\python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [ ]:
##########################################################
#학습
##########################################################
model =  mkmodel()
model.summary()

In [56]:
from sklearn.model_selection import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(train_generator,test_size=0.3)

In [54]:
steps_per_epochs = 30 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 15

history = model.fit( X_train, y_train,
      steps_per_epoch=steps_per_epochs ,
      epochs=epochs )

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'str'>"}), <class 'numpy.ndarray'>